# Load API Keys 

In [5]:
# !pip3 install python-dotenv

In [1]:
from dotenv import load_dotenv

load_dotenv('../env.sh')

True

In [2]:
from langchain.llms import OpenAI

/Users/amrita/PycharmProjects/LLMs/venv/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.13) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


# LangChain 101

## The LLMs

In [3]:
from langchain.llms import OpenAI

In [16]:
llm = OpenAI(model='text-davinci-003', temperature=0.9)

In [17]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))



1. Start your workout routine three times per week with a mid-distance run. Alternate between running a 3 mile route at medium intensity one workout and doing hill sprints for a one mile route the next. 

2. After your run, focus on high-intensity interval training (HIIT) to build up your cardiovascular endurance. Begin with a light warm-up, then do a mixture of squats, lunges, burpees, and jumping jacks for 30 seconds each. Rest for 30 seconds between each exercise.

3. After your HIIT session, cool down with a light jog. Alternatively, you can go for a swim or bike ride to change up your routine and give your muscles a break from running.

4. Twice a week, replace your run or HIIT session with a leisurely bike ride or walk to a nearby park or lake. This will help to build up your endurance and help reduce burnout.

5. Finally, be sure to stretch before and after each workout to improve flexibility and reduce risk of injury.


## The Chain

In [4]:
from langchain import prompts, llms, chains 

In [ ]:
llm = llms.OpenAI(model = "text-davinci-003", temperature = 0.9)
prompt = prompts.PromptTemplate(input_variables = ["product"], template = "What is a good name for a company that makes {product}?")

chain = chains.LLMChain(llm=llm, prompt=prompt)

print(chain.run("eco-friendly water bottles"))

## The Memory

In [5]:
from langchain import memory

In [ ]:
llm = llms.OpenAI(model = "text-davinci-003", temperature = 0)
conversation = chains.ConversationChain(llm=llm, verbose=True, memory=memory.ConversationBufferMemory())

conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

print(conversation)

## Deep Lake Vector store

In [12]:
from langchain.embeddings import openai
from langchain.vectorstores.deeplake import DeepLake
from langchain import text_splitter

In [14]:
llm = llms.OpenAI(model="text-davinci-003", temperature=0)
embeddings = openai.OpenAIEmbeddings(model="text-embedding-ada-002")

In [15]:
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]

text_split = text_splitter.RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_split.create_documents(texts)

In [16]:
my_activeloop_org_id = "amrita95"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"

dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

In [17]:
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Deep Lake Dataset in hub://amrita95/langchain_course_from_zero_to_hero already exists, loading from the storage


In [18]:
db.add_documents(docs)

Dataset(path='hub://amrita95/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
   text       text      (2, 1)      str     None   


['42b14564-a2fc-11ee-a4f3-a683e744668d',
 '42b14636-a2fc-11ee-a4f3-a683e744668d']

In [19]:
retrieval_qa = chains.RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [20]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [21]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.


In [23]:
# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]

docs = text_split.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

/

Dataset(path='hub://amrita95/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['dcf07956-a2fc-11ee-a4f3-a683e744668d',
 'dcf07a6e-a2fc-11ee-a4f3-a683e744668d']

In [24]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...
 I need to find out when Michael Jordan was born.
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.


### Google search Tool

In [25]:
from langchain.llms import OpenAI
from langchain.agents import AgentType, load_tools, initialize_agent, Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [26]:
llm = OpenAI(model="text-davinci-003", temperature=0)

In [28]:
## load Google API keys
load_dotenv('../env.sh')

True

In [30]:
!poetry add google-api-python-client

Using version ^2.111.0 for google-api-python-client

Updating dependencies
Resolving dependencies... (6.4s)s://files.pythonhosted.org/packages/a2/91/2d843adb9fbd911e0da45fbf6f18ca89d07a087c3daa23e955584f90ebf4/cachetools-5.3.2-py3-none-any.whl (2.9s)

Package operations: 11 installs, 0 updates, 0 removals

  • Installing pyasn1 (0.5.1): Pending...
  • Installing pyasn1 (0.5.1): Downloading... 0%
  • Installing pyasn1 (0.5.1): Downloading... 100%
  • Installing pyasn1 (0.5.1): Installing...
  • Installing pyasn1 (0.5.1)
  • Installing cachetools (5.3.2): Pending...
  • Installing pyasn1-modules (0.3.0): Pending...
  • Installing rsa (4.9): Pending...
  • Installing rsa (4.9): Pending...
  • Installing pyasn1-modules (0.3.0): Downloading... 0%
  • Installing rsa (4.9): Pending...
  • Installing rsa (4.9): Pending...
  • Installing pyasn1-modules (0.3.0): Downloading... 100%
  • Installing rsa (4.9): Pending...
  • Installing rsa (4.9): Pending...
  • Installing pyasn1-modules (0.3.0): In

In [31]:
search = GoogleSearchAPIWrapper()

In [32]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

In [33]:
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

In [37]:
response = agent("What's the latest news about the Mars Rover?")
print(response['output'])



> Entering new  chain...
 I should find out what the latest news is.
Action: google-search
Action Input: "Amrita Veerabaghu news"
Observation: Exciting news - we've joined Instagram!… Exciting news - we've joined Instagram!… ... Amrita Sundari Veerabagu. Software Engineer, Machine Learning at Niantic, ... 2013 – Breaking News Live; 2013 - 3 G; 2013 - Last Bus 8:30 PM; 2014 - Law Point ... Amrita TV, Devan, Malayalam. 2010, Indhraneelam, Surya TV, Malayalam. 2010- ... Amrita Singh. Robert Half University of California, Santa Cruz. San Jose ... Concord, CA · Connect · Amrita Sundari Veerabagu. San Jose, CA · Connect · L. Mattock ... Nov 2, 2023 ... News Edition, American Chemical Society, Organic Letters, Organic ... Amrita V.; Kleinheinz, Tracy; Lai, Tommy; Leipold, Douglas D.; Li, Chun ... Premkumar S. Principal Software Engineer at Optum. Optum Amrita School of Engineering,Coimbatore. Chennai, Tamil Nadu, India. 560 followers 500+ connections. Amrita Vidyalayam. 4.248 Ratings. Marav

## Tools in LangChain

### Google Search + Text Summarizer tool

In [39]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [40]:
llm = OpenAI(model="text-davinci-003", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

In [41]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

In [42]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

In [44]:
response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
print(response['output'])



> Entering new  chain...
 I need to find the latest news about the Mars rover and then summarize it.
Action: Search
Action Input: Latest news about the Mars rover
Observation: NASA's Curiosity Mars rover captured these clouds just after sunset on March 19, 2021, ... Feb 8, 2023 ... NASA's Curiosity Mars rover has discovered lots of evidence of ancient lakes on the Red Planet – and this recent panorama shows intriguing ... News ; NASA's Perseverance Rover Deciphers Ancient History of Martian Lake. December 12, 2023 ; NASA's MAVEN Observes the Disappearing Solar Wind. December 11, ... Dec 15, 2021 ... These and other findings were presented today during a news briefing at the American Geophysical Union fall science meeting in New Orleans. Even ... NASA's Mars 2020 Perseverance rover will look for signs of past microbial life, cache rock and soil samples, and prepare for future human exploration. Apr 13, 2023 ... NASA's Curiosity Mars rover will drive faster and reduce wear on its wheel